# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import time
from sagemaker.sklearn import SKLearnModel
import boto3
%matplotlib inline

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
from utils import StarbucksProject

In [ ]:
SP = StarbucksProject()

In [ ]:
SP.fit_etl()
SP.fit_preprocess()
SP.call_model_architecture_tuned()
SP.full_pipeline()

In [1]:
SP.process_model_pipeline

NameError: name 'SP' is not defined

In [6]:
# Create a SageMaker model in order to able to deploy
model = SKLearnModel(
    model_data=f's3://{SP.bucket}/backup/process_model_pipeline.tar.gz',
    role=SP.role,
    image_uri=SP.training_image_uri,
    entry_point = 'utils.py'
)

# Deploy pipeline to an endpoint
predictor = model.deploy(instance_type='ml.m5.large', initial_instance_count = 1, endpoint_name='process-model-pipeline')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
----------!

In [8]:
# Profile EDA

In [9]:
# fig, axs = plt.subplots(2,2)

# sns.kdeplot(profile, hue = 'gender', x = 'age', ax = axs[0][0])
# sns.kdeplot(profile, hue = 'gender', x = 'income', ax = axs[0][1])
# sns.histplot(profile, x = 'gender', ax = axs[1][0])
# sns.histplot(profile, x = 'year', ax = axs[1][1])

# fig.set_figheight(10)
# fig.set_figwidth(14)

# plt.savefig('images/profile_eda.png')

In [10]:
# Offers EDA

In [11]:
# df_offers = transcript.dropna(subset=['offer_id']).drop(['reward'], axis = 1).merge(portfolio.drop(['channels'], axis = 1), left_on = 'offer_id', right_on = 'id', how = 'inner')

# fig, axs = plt.subplots(2,2)

# sns.histplot(df_offers, x = 'reward', ax = axs[0][0])
# sns.histplot(df_offers, x = 'difficulty', ax = axs[0][1])
# sns.histplot(df_offers, x = 'duration', ax = axs[1][0])
# sns.histplot(df_offers, x = 'offer_type', ax = axs[1][1])
# fig.set_figheight(10)
# fig.set_figwidth(14)

# plt.savefig('images/offers_eda.png')

In [12]:
SP.complete_table.head()

,customer_id,offer_id,time_receive,time_view,time_completion,time_for_completion,time_for_view,general_journey,amount,gender,...,month,day,reward,difficulty,duration,offer_type,email,mobile,social,web
0,7ec292cf2eeb41288582c056fbe7775d,3f207df678b143eea3cee63160fa8bed,408,414.0,None,None,6.0,offer_viewed,0.00,F,...,8,7,0.0,0.0,96.0,informational,1.0,1.0,0.0,1.0
1,b3ad8755d0ac47faa6e9c10954fec6a4,no_offer,696,696.0,696,0,0.0,transactioned,14.77,F,...,8,27,0.0,0.0,0.0,no_offer,0.0,0.0,0.0,0.0
2,fa78c77815f14f7e973c43c229ae3556,2906b810c7d4411798c6938adc9daaa5,168,NaN,None,None,NaN,offer_received,0.00,M,...,5,18,2.0,10.0,168.0,discount,1.0,1.0,0.0,1.0
3,ce2a843de1684511a4276942c4ccead8,4d5c57ea9a6940dd891ad53e9dbe8da0,576,588.0,None,None,12.0,offer_viewed,0.00,F,...,10,31,10.0,10.0,120.0,bogo,1.0,1.0,1.0,1.0
4,38a9ec4b59a549b786ae49c5af104c7f,no_offer,438,438.0,438,0,0.0,transactioned,7.35,M,...,8,1,0.0,0.0,0.0,no_offer,0.0,0.0,0.0,0.0


In [21]:
SP.complete_table[SP.complete_table.offer_type == 'bogo']

,customer_id,offer_id,time_receive,time_view,time_completion,time_for_completion,time_for_view,general_journey,amount,gender,...,month,day,reward,difficulty,duration,offer_type,email,mobile,social,web
3,ce2a843de1684511a4276942c4ccead8,4d5c57ea9a6940dd891ad53e9dbe8da0,576,588.0,None,None,12.0,offer_viewed,0.0,F,...,10,31,10.0,10.0,120.0,bogo,1.0,1.0,1.0,1.0
12,f6540770edbb4de589eb5d0c9054e99a,4d5c57ea9a6940dd891ad53e9dbe8da0,408,414.0,438,30,24.0,offer_completed,10.0,M,...,12,9,10.0,10.0,120.0,bogo,1.0,1.0,1.0,1.0
20,379c3d188896452bb3ce05c91801e191,ae264e3637204a6fb9bb56bc8210ddfd,0,NaN,132,132,NaN,offer_completed,10.0,M,...,12,10,10.0,10.0,168.0,bogo,1.0,1.0,1.0,0.0
22,8e0f512b4278472782bd4e1154cfa3b5,f19421c1d4aa40978ebb69ca19b0e20d,576,576.0,None,None,0.0,offer_viewed,0.0,None,...,6,27,5.0,5.0,120.0,bogo,1.0,1.0,1.0,1.0
23,b2f1e283593d4c7182185e4bb3320de0,f19421c1d4aa40978ebb69ca19b0e20d,504,516.0,None,None,12.0,offer_viewed,0.0,F,...,6,10,5.0,5.0,120.0,bogo,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215900,4c75bfae0f064ea3a28b13c90e6d362f,ae264e3637204a6fb9bb56bc8210ddfd,336,NaN,348,12,NaN,offer_completed,10.0,F,...,11,1,10.0,10.0,168.0,bogo,1.0,1.0,1.0,0.0
215904,2eebc59bf4a64f83974dfe044be368a2,4d5c57ea9a6940dd891ad53e9dbe8da0,576,624.0,None,None,48.0,offer_viewed,0.0,F,...,7,7,10.0,10.0,120.0,bogo,1.0,1.0,1.0,1.0
215906,b99c01e940364ecdbef03dae24c6554b,4d5c57ea9a6940dd891ad53e9dbe8da0,0,0.0,None,None,0.0,offer_viewed,0.0,M,...,7,23,10.0,10.0,120.0,bogo,1.0,1.0,1.0,1.0
215908,1ea7547159ae462192c3576d258c2bb7,f19421c1d4aa40978ebb69ca19b0e20d,504,504.0,504,0,0.0,offer_completed,5.0,M,...,8,26,5.0,5.0,120.0,bogo,1.0,1.0,1.0,1.0
